In [7]:
url = 'http://187.111.99.18'
port = '9004'

In [ ]:
query = {'CODE': '313'}

res = requests.get(f'{url}:{port}', query, stream=True, timeout=None)

# Event Expiration Time (Max-Age) Testing

#### API project directory

In [1]:
cd ../../Apps/APIs/octa-api

C:\Users\luisr\Desktop\Repositories\Apps\APIs\octa-api


#### API request class

In [2]:
import requests, pandas as pd

class api:
    
    localurl='http://127.0.0.1:5000'
    baseurl='https://api.octacity.dev'
        
    def df(subpath, query={}, local=True):
        url = api.localurl if local else api.baseurl
        return pd.DataFrame(requests.get(f'{url}/{subpath}', params=query).json())

C:\Users\luisr\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### Events and polygons' events count

In [4]:
waze = api.df('waze/alerts', local=False)
waze = waze[waze['subtype']=='HAZARD_WEATHER_FLOOD']

comando = api.df('comando/events', local=False)
comando = comando[comando['pop_id']==5]

print(f"BOLSÃO: {len(comando)} · WAZE-FLOOD: {len(waze)}")
print(f"BOLSÃO-POLY: {len(comando[comando['cluster_id']!=-1])} · WAZE-FLOOD-POLY: {len(waze[(waze['cluster_id']!=-1)])}")

BOLSÃO: 1 · WAZE-FLOOD: 20
BOLSÃO-POLY: 1 · WAZE-FLOOD-POLY: 0


#### Polygons' events count (max-age applied)

In [6]:
polygons = api.df('polygons', local=False)

bolsao = polygons['bolsão_status'].sum()
waze_flood = polygons['waze_flood_status'].sum()

print(f'BOLSÃO-POLY: {bolsao} · WAZE-FLOOD-POLY: {waze_flood}')

BOLSÃO-POLY: 0 · WAZE-FLOOD-POLY: 0


If result above is different from previous result, it means that max_age filter applies for the missing events in polygons result

#### Testing response datetime format

In [42]:
from datetime import datetime as dt
import pytz; tz_br = pytz.timezone('Brazil/East')

In [43]:
dt.now(tz_br) - pd.to_datetime(waze['pubMillis']).dt.tz_localize('Brazil/East')

4     0 days 00:00:27.033841
8     0 days 00:01:05.033841
12    0 days 00:01:32.033841
32    0 days 00:05:20.033841
78    0 days 00:13:23.033841
95    0 days 00:16:26.033841
116   0 days 00:20:28.033841
121   0 days 00:22:17.033841
151   0 days 00:31:44.033841
163   0 days 00:50:54.033841
168   0 days 01:04:41.033841
Name: pubMillis, dtype: timedelta64[ns]